In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
!pip install pandas
!pip install numpy
import pandas as pd
import numpy as np

In [96]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Train_samsung.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Test_samsung_noclass.csv')

In [97]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      479 non-null    object 
 1   X2      490 non-null    object 
 2   X3      480 non-null    object 
 3   X4      491 non-null    object 
 4   X5      467 non-null    object 
 5   X6      491 non-null    int64  
 6   X7      491 non-null    float64
 7   X8      475 non-null    float64
 8   X9      479 non-null    float64
 9   X10     451 non-null    float64
 10  X11     491 non-null    object 
 11  Class   491 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 46.2+ KB


In [98]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      122 non-null    object 
 1   X2      121 non-null    object 
 2   X3      119 non-null    object 
 3   X4      123 non-null    object 
 4   X5      115 non-null    object 
 5   X6      123 non-null    int64  
 6   X7      123 non-null    float64
 7   X8      117 non-null    float64
 8   X9      121 non-null    float64
 9   X10     113 non-null    float64
 10  X11     123 non-null    object 
dtypes: float64(4), int64(1), object(6)
memory usage: 10.7+ KB


In [99]:
train.isna().sum()

X1       12
X2        1
X3       11
X4        0
X5       24
X6        0
X7        0
X8       16
X9       12
X10      40
X11       0
Class     0
dtype: int64

In [100]:
test.isna().sum()

X1      1
X2      2
X3      4
X4      0
X5      8
X6      0
X7      0
X8      6
X9      2
X10    10
X11     0
dtype: int64

In [101]:
train['X1'].fillna(train['X1'].mode()[0], inplace=True)
train['X2'].fillna(train['X2'].mode()[0], inplace=True)
train['X5'].fillna(train['X5'].mode()[0], inplace=True)
train['X3'].fillna(train['X3'].mode()[0], inplace=True)

train['X8'].fillna(round(train['X8'].mean()), inplace=True)
train['X9'].fillna(round(train['X9'].mean()), inplace=True)
train['X10'].fillna(round(train['X10'].mean()), inplace=True)

test['X1'].fillna(train['X1'].mode()[0], inplace=True)
test['X2'].fillna(train['X2'].mode()[0], inplace=True)
test['X5'].fillna(train['X5'].mode()[0], inplace=True)
test['X3'].fillna(train['X3'].mode()[0], inplace=True)

test['X8'].fillna(round(train['X8'].mean()), inplace=True)
test['X9'].fillna(round(train['X9'].mean()), inplace=True)
test['X10'].fillna(round(train['X10'].mean()), inplace=True)

In [102]:
!pip install lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [103]:
X_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]

# Chia tập train thành train và valid để đánh giá mô hình
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Sử dụng LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train_split, X_valid_split, y_train_split, y_valid_split)

# In ra kết quả
print(models)

100%|██████████| 29/29 [00:03<00:00,  7.55it/s]

[LightGBM] [Info] Number of positive: 276, number of negative: 116
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 331
[LightGBM] [Info] Number of data points in the train set: 392, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.704082 -> initscore=0.866811
[LightGBM] [Info] Start training from score 0.866811
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [104]:
!pip install scikit-learn
from sklearn.ensemble import AdaBoostClassifier

In [105]:
X_train = pd.get_dummies(X_train)

best_model = AdaBoostClassifier()

# Train the model on the entire training set
best_model.fit(X_train, y_train)

# Apply the same one-hot encoding to the test data
test_data = pd.get_dummies(test)

# Ensure all columns in training data are also in test data
missing_cols = set(X_train.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0
test_data = test_data[X_train.columns]

# Now predict using the preprocessed test data
test_predictions = best_model.predict(test_data)

# Save results
test['Predicted_label'] = test_predictions
test.to_csv('Test_samsung_with_labels.csv', index=False)

In [106]:
test

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Predicted_label
0,Male,Yes,0,Graduate,No,9083,0.00,228.00,360.00,1.00,Semiurban,Y
1,Male,Yes,0,Graduate,No,4310,0.00,130.00,360.00,1.00,Semiurban,Y
2,Male,Yes,2,Graduate,No,4167,1447.00,158.00,360.00,1.00,Rural,Y
3,Female,No,0,Graduate,No,2900,0.00,71.00,360.00,1.00,Rural,N
4,Male,No,0,Graduate,No,2718,0.00,70.00,360.00,1.00,Semiurban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
118,Male,Yes,0,Graduate,No,3716,0.00,42.00,180.00,1.00,Rural,Y
119,Female,No,0,Graduate,No,2507,0.00,56.00,360.00,1.00,Rural,Y
120,Male,Yes,1,Graduate,No,2882,1843.00,123.00,480.00,1.00,Semiurban,N
121,Male,Yes,2,Graduate,No,2500,1840.00,109.00,360.00,1.00,Urban,Y
